# __Veri Temizleme__

Veriler hatalı, eksik, düzesiz bir şekilde elimizde bulunur. Bu verilerin uygun şekilde temizlenmesi gerekir ki Makine Öğrenmesi uygulamalarında kullanabilelim

> Düzensiz veriye sahip bir daataframe oluşturalım

In [49]:
import pandas as pd
import numpy as np

df=pd.DataFrame({
    "From_To" : ["LoNDon_paris","MAdrid_miLAN", "loNDon_STockhOLm","BudaPESt_lonDON","BRUSselS_loNDon"],
    "FlightNumber" : [10045,np.NaN,10065,np.NaN,10085],
    "RecentDelays" : [[23,47],[],[24,43,87],[13],[67,32]],
    "Airline" : ["KLM(!)","<Air France>(12)","(British Airways.)","12. Air France",'"Swiss Air"']
})
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France>(12)
2,loNDon_STockhOLm,10065.0,"[24, 43, 87]",(British Airways.)
3,BudaPESt_lonDON,NaN,[13],12. Air France
4,BRUSselS_loNDon,10085.0,"[67, 32]","""Swiss Air"""


#### __1.__ FlightNumber sütununda bazı değerler eksik. Bu sayılar her satırda 10 artacak şekilde düzenlenmeli. Eksi olan verileri dolduralım ve sütun tam sayı sütunu olsun

> - __interpolate__ ile eksik veri düzeltme yapılır. interpolate ile belirttiğimiz sütuna bakar ve NaN verileri verilerden bir anlam çıkararak doldurur. 

In [50]:
df["FlightNumber"] = df["FlightNumber"].interpolate()
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055.0,[],<Air France>(12)
2,loNDon_STockhOLm,10065.0,"[24, 43, 87]",(British Airways.)
3,BudaPESt_lonDON,10075.0,[13],12. Air France
4,BRUSselS_loNDon,10085.0,"[67, 32]","""Swiss Air"""


>- __astype(int)__ ile değerleri tamsayı yaparız

In [51]:
df["FlightNumber"] = df["FlightNumber"].interpolate().astype(int)
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055,[],<Air France>(12)
2,loNDon_STockhOLm,10065,"[24, 43, 87]",(British Airways.)
3,BudaPESt_lonDON,10075,[13],12. Air France
4,BRUSselS_loNDon,10085,"[67, 32]","""Swiss Air"""


#### __1.__ From_To sütunu iki ayrı sütun olsa daha anlamlı bir tablo olurdu. Bunun için geçici bir dataFrame oluşturup sütunları ayıralım.

> - __str.split("_")__ ile metinler _'den ayrılır

In [52]:
df.From_To.str.split("_")

0        [LoNDon, paris]
1        [MAdrid, miLAN]
2    [loNDon, STockhOLm]
3     [BudaPESt, lonDON]
4     [BRUSselS, loNDon]
Name: From_To, dtype: object

__str.split("_",expand=True)__ yaptığımızda sütunlara ayrılır

In [53]:
df.From_To.str.split("_",expand=True)

,0,1
0,LoNDon,paris
1,MAdrid,miLAN
2,loNDon,STockhOLm
3,BudaPESt,lonDON
4,BRUSselS,loNDon


> __.__ gecici adında bir df oluşturup bu sütunları bu gecici df'ye alalım ve sütun isimlerini verelim

In [54]:
gecici = df.From_To.str.split("_",expand=True)
gecici.columns=["From","To"]
gecici

,From,To
0,LoNDon,paris
1,MAdrid,miLAN
2,loNDon,STockhOLm
3,BudaPESt,lonDON
4,BRUSselS,loNDon


#### __2.__ "gecici" df'de şehir isimlerini yalnızca ilk harfleri büyük diğerleri küçük olacak şekilde düzeltelim
> .__.str.capitalize()__ ile bu istedğimizi gerçekleştirebiliriz

In [55]:
gecici["From"] = gecici["From"].str.capitalize()
gecici["To"] = gecici["To"].str.capitalize()
gecici

,From,To
0,London,Paris
1,Madrid,Milan
2,London,Stockholm
3,Budapest,London
4,Brussels,London


#### __3.__ "From_To" sütununu "df"den silip "gecici"'y iekleyelim'

> - __drop__ ile sütun silinir. axis=1 verirsek sütun silecektir.

In [56]:
df=df.drop("From_To",axis=1)
df

,FlightNumber,RecentDelays,Airline
0,10045,"[23, 47]",KLM(!)
1,10055,[],<Air France>(12)
2,10065,"[24, 43, 87]",(British Airways.)
3,10075,[13],12. Air France
4,10085,"[67, 32]","""Swiss Air"""


> - __join__ ile istedğimiz verisetini bu verisetine ekleyebiliriz.

In [57]:
df=df.join(gecici)
df

,FlightNumber,RecentDelays,Airline,From,To
0,10045,"[23, 47]",KLM(!),London,Paris
1,10055,[],<Air France>(12),Madrid,Milan
2,10065,"[24, 43, 87]",(British Airways.),London,Stockholm
3,10075,[13],12. Air France,Budapest,London
4,10085,"[67, 32]","""Swiss Air""",Brussels,London


#### __3.__ "Airline" sütununda bazı işaretler ve semboller var. Sadece Hava yolu ismi olmalı

> - __str.extract()__ kullanarak sembollerden kurtuluruz. Bunun için regex kullanırız. Sadece harflerin kullanılması regexi:"a-zA-Z\s" şeklindedir.

In [59]:
df["Airline"]=df["Airline"].str.extract('([ a-zA-Z\s]+)', expand=False)
df

,FlightNumber,RecentDelays,Airline,From,To
0,10045,"[23, 47]",KLM,London,Paris
1,10055,[],Air France,Madrid,Milan
2,10065,"[24, 43, 87]",British Airways,London,Stockholm
3,10075,[13],Air France,Budapest,London
4,10085,"[67, 32]",Swiss Air,Brussels,London


#### __3.__ "RecentDays" sütununda değerler list şeklinde girilmiş. Her bir değer kendi sütununda olmalı ve olmayan değerler ilgili sütunda NaN olmalıdır.

> - "RecentDelays" verileri bir Series olarak alalım ve columns oluştururken column sayısı kadar oluşturalım. Series olan verilerde boş alanlar zaten default olarak NaN olacaktır.
<br> Sütunları delay1,delay2... şeklinde isimlendirelim : 
<br>__"delay_{}".format(n) for n in range(1,len(delays.columns)+1).format(n)__ kullanarak bu işlemi yapabiliriz. (İsimlendirmeyi 0 değil 1'den başlatalım.)
<br> Aslında isimlendirmeyi __delays.columns = ["delay_1","delay_2","delay_3"]__ şeklinde de yapabilirdik. Ancak 10'larca sütun olan bir yapı olduğunu düşündüğümüzde bu zor bir işlem olacaktır. Bu yüzdeb bu tarz bir isimlendirmeyi yapmak işimizi oldukça kolaylaştıracaktır

In [61]:
delays = df["RecentDelays"].apply(pd.Series)
delays

,0,1,2
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


In [63]:
delays.columns = ["delay_{}".format(n) for n in range(1,len(delays.columns)+1)]
delays

,delay_1,delay_2,delay_3
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


> - Şimdi RecentDelay'i silip delaysi df'e ekleyelim

In [64]:
df= df.drop("RecentDelays",axis=1)
df

,FlightNumber,Airline,From,To
0,10045,KLM,London,Paris
1,10055,Air France,Madrid,Milan
2,10065,British Airways,London,Stockholm
3,10075,Air France,Budapest,London
4,10085,Swiss Air,Brussels,London


In [66]:
df=df.join(delays)
df

,FlightNumber,Airline,From,To,delay_1,delay_2,delay_3
0,10045,KLM,London,Paris,23.0,47.0,NaN
1,10055,Air France,Madrid,Milan,NaN,NaN,NaN
2,10065,British Airways,London,Stockholm,24.0,43.0,87.0
3,10075,Air France,Budapest,London,13.0,NaN,NaN
4,10085,Swiss Air,Brussels,London,67.0,32.0,NaN
